# Import Libraries

In [ ]:
# main libraries
import os            # For file and directory management.
import pandas as pd  # For handling structured data in DataFrames.
import numpy as np   # For numerical computations.

# visualization libraries
import matplotlib.pyplot as plt      # For creating visualizations.
import seaborn as sns                # For statistical data visualization.
import missingno as msno             # To check null values existence.
from matplotlib.colors import LinearSegmentedColormap # To define custom colormaps with smooth color transitions.
from matplotlib.colors import ListedColormap          # For creating colormaps from a list of colors.
import warnings            # To manage warning messages.
from PIL import Image      # To handle image loading and manipulation.

# preprocessing libraries
import tensorflow as tf    # For building and training the model.
from sklearn.model_selection import train_test_split    # For splitting datasets.
from tensorflow.keras.utils import to_categorical       # To handle categorical data.
from sklearn.preprocessing import LabelEncoder, MinMaxScaler  # For encoding categorical labels into numeric format.

# Model libraries
from tensorflow.keras.models import Sequential          # For model architecture.
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input, Add, GlobalAveragePooling2D , Concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau   # To optimize model training
from tensorflow.keras import regularizers         # For adding regularization to prevent overfittin
import albumentations as A                        # For image augmentation
from collections import Counter                   # To count the occurrences of labels in the dataset.
from tensorflow.keras.regularizers import l2      # To add L2 weight regularization to layers.
from tensorflow.keras.models import Model

# evaluation libraries
from sklearn.metrics import classification_report , confusion_matrix    # For displaying the model evaluation report
from sklearn.metrics import precision_score, recall_score, f1_score     # To assess the performance of the model

# Read Data

In [ ]:
# Loading the CSV file into a pandas DataFrame.
imgdata = pd.read_csv('/kaggle/input/skin-cancer-mnist-ham10000/hmnist_28_28_RGB.csv')
imgdata

In [ ]:
# Reading the metadata CSV file into a pandas DataFrame.
fdata = pd.read_csv('/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv')
fdata

In [ ]:
# Display the frequency of unique values in the 'dx' column of the `fdata` DataFrame.
print(fdata['dx'].value_counts())

# Display the frequency of unique values in the 'label' column of the `imgdata` DataFrame.
print(imgdata['label'].value_counts())

In [ ]:
# Displaying a concise summary of the dataset.
# The output includes the following information:
# 1. Column names and their data types (e.g., int, float, object).
# 2. Non-null counts for each column, which helps in identifying missing data.
# 3. Memory usage of the DataFrame.
# This is for understanding the structure of the dataset and preparing for preprocessing.
print(fdata.info())

**Comments:**
 1. The dataset contains 10015 instances.
 2. There are null values in "age" column.
 3. All columns are Categorical except "age" Column.

In [ ]:
# To check if there is at least one missing value in each column.
# This outputs a Series where the index corresponds to the column names, and the values are True (if the column has missing data) or False (if it doesn't).
fdata.isna().any()

In [ ]:
# Create a visualizing matrix plot to display the missing data in the dataset.
# show a heatmap-like representation where:
# - Each row corresponds to a record (e.g., data point) in the dataset.
# - Each column represents a feature (e.g., a column in the dataset).
# - White gaps indicate missing values, making it easy to spot incomplete data.
msno.matrix(fdata)

# Exploratory Data Analysis (EDA)

**We need to answer the following questions:**
  1. What is the frequency of different lesion types present in the dataset?
  2. What is the type of the distribution of lesion localization across various anatomical sites?
  3. What is the proportion of patients according to the gender?
  4. How are patients distributed according to the age?

In [ ]:
# - Ignoring FutureWarnings to ensure cleaner outputs during development.
warnings.filterwarnings("ignore", category=FutureWarning)
data = fdata
# Ensures the use of infinity values is not treated as NaN (Not a Number) when working with datasets.
pd.options.mode.use_inf_as_na = False

# Defining a custom color palette for visualizations:
custom_palette = ['#493628', '#AB886D', '#D6C0B3', '#E4E0E1', '#E7E8D8','#CADABF', '#B5CFB7', '#BC9F8B', '#705C53', '#B7B7B7']

# Creating a grid of subplots with a figure size of 15x10 inches:
# The grid has 2 rows and 2 columns, allowing space for multiple visualizations.
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# count plot to visualize the distribution of lesion types:
# - Data: `data` DataFrame, expected to have a column 'dx' containing lesion type labels.
# - x='dx': Specifies that the x-axis represents lesion types.
# - ax=axes[0, 0]: Places the plot in the first subplot (top-left corner in this case).
# - palette=custom_palette: Applies the defined color palette for better aesthetics.
# - order=data['dx'].value_counts().index: Ensures the lesion types are displayed in descending order of frequency.
sns.countplot(data=data, x='dx', ax=axes[0, 0],palette=custom_palette[:len(data['dx'].unique())],order=data['dx'].value_counts().index)

#--------------------------------------------------------------------------------------------------------------------------------------------

# Count plot for the count of different lesion localizations in the dataset.
# - sns.countplot: Creates a bar plot for categorical data.
# - data=data: Specifies the dataset to be used.
# - y='localization': Indicates the column representing lesion localization to be plotted along the y-axis.
# - ax=axes[0, 1]: Specifies the subplot location in a grid of plots (top-right corner in this case).
# - palette=custom_palette[:len(data['localization'].unique())]: Sets a custom color palette, with unique colors corresponding to the number of unique localization categories.
# - order=data['localization'].value_counts().index: Orders the categories by their count in descending order.
sns.countplot(data=data,y='localization',ax=axes[0, 1],palette=custom_palette[:len(data['localization'].unique())],order=data['localization'].value_counts().index)

#--------------------------------------------------------------------------------------------------------------------------------------------
# This pie chart shows the distribution of gender in the dataset.
gender_counts = data['sex'].value_counts()     # - data['sex'].value_counts(): Counts the occurrences of each gender in the dataset.

# - labels=gender_counts.index: Sets the pie chart labels as the unique genders.
# - autopct='%1.1f%%': Displays the percentage distribution with one decimal place.
# - startangle=90: Starts the pie chart at 90 degrees for better orientation.
# - colors=custom_palette[:len(gender_counts)]: Uses the custom palette for consistent coloring.
axes[1, 0].pie(gender_counts,labels=gender_counts.index,autopct='%1.1f%%',startangle=90,colors=custom_palette[:len(gender_counts)])
axes[1, 0].set_title('Gender Distribution')   # - set_title('Gender Distribution'): Sets the title of the subplot.
#--------------------------------------------------------------------------------------------------------------------------------------------
# This histogram is for the distribution of patients' ages.
# - sns.histplot(): Plots the histogram for numerical data.
# - data['age']: Specifies the age column from the dataset.
# - ax=axes[1, 1]: Specifies the subplot location in a grid of plots (bottom-right corner).
# - kde=True: Adds a Kernel Density Estimate (KDE) curve for a smooth representation of the data distribution.
# - bins=30: Divides the data range into 30 intervals for the histogram.
# - color=custom_palette[0]: Sets the bar color to the first color in the custom palette.
sns.histplot(data['age'],ax=axes[1, 1],kde=True,bins=30,color=custom_palette[0])

#------------------------------------------------------------------------------------------------------------------------------------------
# Setting titles for subplots to provide context to each visualization:
# The first subplot (top-left, indexed as [0, 0]) is titled 'Lesion Types'
axes[0, 0].set_title('Lesion Types')

# The second subplot (top-right, indexed as [0, 1]) is titled 'Localization',
axes[0, 1].set_title('Localization')

# The third subplot (bottom-right, indexed as [1, 1]) is titled 'Age Distribution',
axes[1, 1].set_title('Age Distribution')

# Adjusting subplot spacing to ensure that titles and content are displayed without overlap.
plt.tight_layout()
plt.show()  # Displaying the plots.

**We need to answer the following question:**
 1. How does age vary across different lesion types? And are there outliers or not?
 2. How is the distribution of diagnosis based on gender?
 3. What is the common diagnosis in each gender?
 4. How are lesion types distributed across body locations?
 5. How many are diagnosis methods used for each lesion type? or How are methods more commonly used for specific 
    diagnosis?

In [ ]:
# Define a custom color palette with of beige and brown for visual consistency in plots.
beige_brown_palette = ["#D2B48C", "#C2B280", "#8B4513", "#6F4F28", "#3E2B1B"]
df= data
# Create a figure with a grid of subplots (2 rows and 2 columns) for multiple visualizations.
# The figure size is set to (15, 10) for better readability.
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Boxplot for the age distribution across different types of skin lesions.
# - 'dx' represents the diagnosis type (e.g., melanoma, benign, etc.).
# - 'age' is the dependent variable.
# - The custom palette is applied for enhancing appearance.
# - The plot is placed in the first subplot (top-left)
sns.boxplot(x='dx', y='age', data=df, palette=beige_brown_palette, ax=axes[0, 0])
axes[0, 0].set_title('Age Distribution by Skin Lesion Type')   # Setting the title for the subplot.

# Plot 2: Countplot to show the frequency of diagnoses by gender.
# - 'dx' is the x-axis, and gender ('sex') is used as the hue to differentiate groups.
# - Custom palette is applied, and the plot is placed in the top-right subplot.
sns.countplot(x='dx', hue='sex', data=df, palette=beige_brown_palette, ax=axes[0, 1])
axes[0, 1].set_title('Diagnosis by Gender')  # Set the title for the plot.
axes[0, 1].set_xticklabels(axes[0, 1].get_xticklabels(), rotation=45)  # Rotate x-axis labels for better visibility.

# Plot 3: Heatmap to show the relationship between lesion localization and diagnosis type.
# - A crosstab table is created to aggregate the data, and annotations are added to the heatmap for clarity.
# - A custom colormap is derived from the beige-brown palette.
# - The plot is placed in the bottom-left subplot.
cmap = ListedColormap(beige_brown_palette)  # Create a colormap for the heatmap.
sns.heatmap(pd.crosstab(df['localization'], df['dx']), annot=True, fmt="d", cmap=cmap, ax=axes[1, 0])
axes[1, 0].set_title('Localization vs Diagnosis')  # Set the title for the plot.

# Plot 4: Grouped bar chart to show the count of diagnosis types vs diagnosis methods.
# - A pivot table is created to summarize the data for plotting.
# - 'dx' is used as the x-axis, and the pivot table values are grouped and plotted.
# - The custom colormap is applied, and the plot is placed in the bottom-right subplot.
pivot = df.pivot_table(index='dx', columns='dx_type', aggfunc='size', fill_value=0)  # Create a pivot table.
pivot = pivot.reset_index()  # Reset the index for compatibility with plotting.

pivot.plot(x='dx', kind='bar', figsize=(12, 8), colormap=cmap, ax=axes[1, 1])   # Plot as a bar chart.
axes[1, 1].set_title('Grouped Bar Chart: Diagnosis Type vs Diagnosis Method')   # Set the title for the plot.
axes[1, 1].set_ylabel('Count')  # Label the y-axis.
axes[1, 1].set_xlabel('Diagnosis Type (dx)')   # Label the x-axis.
axes[1, 1].set_xticklabels(axes[1, 1].get_xticklabels(), rotation=45)  # Rotate x-axis labels for better visibility.

plt.tight_layout()  # Adjust the layout to prevent overlapping of subplots.
plt.show()  # Display all the plots.

## Displaying five randomly selected images from each diagnosis.

In [ ]:
# Displaying summary statistics for the DataFrame 'fdata', including count, mean, standard deviation, min, and max for each numerical column
fdata.describe()

In [ ]:
# Defining paths to the two parts of the image dataset and the metadata file.
images_path_1 = '/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_1/'
images_path_2 = '/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_images_part_2/'
metadata_path = '/kaggle/input/skin-cancer-mnist-ham10000/HAM10000_metadata.csv'

df = pd.read_csv(metadata_path)

# Adding a new column 'image_path' to the DataFrame that contains the full path to the corresponding image file by:
# - Checking if the image exists in 'images_path_1'.
# - If not, it assigns the path from 'images_path_2'.
df['image_path'] = df['image_id'].apply(
    lambda x: os.path.join(images_path_1, x + '.jpg') if os.path.exists(os.path.join(images_path_1, x + '.jpg')) 
    else os.path.join(images_path_2, x + '.jpg')
)

# Extracting the unique categories (diagnosis labels) from the dataset.
# The 'dx' column in the metadata contains the labels.
categories = df['dx'].unique()
num_samples = 5  # Defining the number of sample images to display per category.
# Creating a grid of subplots to display sample images from each category.
# The number of rows is equal to the number of categories, and each row has 'num_samples' images.
fig, axes = plt.subplots(len(categories), num_samples, figsize=(15, 3 * len(categories)))

# Iterating over each category to display sample images.
for i, category in enumerate(categories):
    # Randomly sampling 'num_samples' images from the current category.
    category_images = df[df['dx'] == category].sample(num_samples)

    # Iterating over the sampled images to display them in the corresponding subplot.
    for j, image_path in enumerate(category_images['image_path']):        
        img = Image.open(image_path)   # Loading the image from the file path.
        axes[i, j].imshow(img)         # Displaying the image in the corresponding subplot.
        axes[i, j].set_title(category) # Setting the title of the subplot to the category label.
        axes[i, j].axis('off')         # Removing axis ticks for a cleaner display.

plt.tight_layout()   # Adjusting the layout to prevent overlapping of subplots.
plt.show()           # Displaying the grid of images.

## Displaying five randomly selected images from the dataset, categorized by gender.

In [ ]:
# Filtering the dataset to separate images based on the 'sex' column.
# The variable 'males' contains rows where the 'sex' column is 'male'.
# The variable 'females' contains rows where the 'sex' column is 'female'.
males = df[df['sex'] == 'male'] 
females = df[df['sex'] == 'female']  

# Visualizing 5 images of male subjects from the dataset.
plt.figure(figsize=(15, 5)) # Setting the figure size for better visualization.

# Looping through the first 5 male entries.
for i in range(5):  
    image_path = males.iloc[i]['image_path']  # Extracting the image path for each male subject.
    image = Image.open(image_path)   # Opening the image using PIL Library. 
    plt.subplot(1, 5, i + 1)         # Creating a subplot for each image (1 row, 5 columns).
    plt.imshow(image)  # Displaying the image.
    plt.axis('off')    # Hiding the axes for a cleaner look.
    plt.title(f'Male: {males.iloc[i]["dx"]}') # Setting the title with the diagnosis ('dx') for each image.
plt.show()    # Displaying the figure.

# Visualizing 5 images of female subjects from the dataset.
plt.figure(figsize=(15, 5))  # Setting the figure size for better visualization.
for i in range(5):   # Looping through the first 5 female entries.
    image_path = females.iloc[i]['image_path']    # Extracting the image path for each female subject.
    image = Image.open(image_path)  # Opening the image using PIL Library.
    plt.subplot(1, 5, i + 1)   # Creating a subplot for each image (1 row, 5 columns).
    plt.imshow(image)   # Displaying the image.
    plt.axis('off')     # Hiding the axes for a cleaner look.
    plt.title(f'Female: {females.iloc[i]["dx"]}') # Setting the title with the diagnosis ('dx') for each image.
plt.show()    # Displaying the figure.

In [ ]:
# Visualizing the class distribution using a pie chart
# Create a figure with a size of 6x6 inches.
plt.figure(figsize=(6, 6))

# Plot a pie chart using the counts of each label from the 'label' column in the imgdata DataFrame.
# Display percentages on the chart with one decimal point for each slice.
plt.pie(imgdata['label'].value_counts().values , labels=imgdata['label'].value_counts().index ,autopct='%1.1f%%')
plt.title('Classes pie chart')    # Add a title to the chart: "Classes pie chart".
plt.show()   # Show the chart

# Preprocessing

In [ ]:
# drop unnececcary columns
fdata = fdata.drop(columns = ['lesion_id' , 'image_id' ,'dx'] , axis =1)
fdata.head()    # Display the first rows

In [ ]:
# fill nulls in age columns using its mean value
fdata['age'] = fdata['age'].fillna(51)

# Split

In [ ]:
# split to features & target
y = imgdata['label']

In [ ]:
# drop target column to isolate the feature data
imgs = imgdata.drop(columns = ['label'] , axis =1)

In [ ]:
# concatenate pixels value with row data
data = pd.concat([fdata, pd.DataFrame(imgs)], axis=1)
data

In [ ]:
# Split the dataset into 80% training data and 20% testing data.
x_trainall, x_testall, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=42, stratify=y)

# Split the training data into 80% training data and 20% validation sets.
x_trainall, x_valall, y_train, y_val = train_test_split(x_trainall, y_train, test_size=0.2, random_state=42, stratify=y_train)

In [ ]:
# Display the column names of the x_trainall DataFrame
x_trainall.columns

In [ ]:
# Display class distribution after splitting the dataset
print("\nClass distribution after splitting:")

# Print the distribution of classes in the training set
print("Training set distribution:")
print(y_train.value_counts().sort_index())

# Print the distribution of classes in the validation set
print("\nValidation set distribution:")
print(y_val.value_counts().sort_index())

# Print the distribution of classes in the test set
print("\nTest set distribution:")
print(y_test.value_counts().sort_index())

# Display the number of samples in each dataset (training, validation, test)
print("\nNumber of samples in each set:")  
print(f"Training: {len(y_train)}")         # Number of samples in the training set
print(f"Validation: {len(y_val)}")         # Number of samples in the validation set
print(f"Test: {len(y_test)}")              # Number of samples in the test set

In [ ]:
# Display the column names of the x_trainall DataFrame
x_trainall.columns

In [ ]:
# split into images and row data for training dataset

# Select specific columns from the dataset to create the 'x_train_tabilar' dataframe
x_train_tabilar = x_trainall[['dx_type', 'age', 'sex', 'localization']]

# Drop the selected columns from the original dataset to create 'x_train_imgs' containing only image data
x_train_imgs = x_trainall.drop(columns= ['dx_type', 'age', 'sex', 'localization'] , axis =1)
x_train_tabilar

In [ ]:
# Split into image and tabular data for validation test

# Extracting the tabular features (dx_type, age, sex, localization) from validation dataset
X_tabular_val = x_valall[['dx_type','age', 'sex' , 'localization']]

# Extracting the image features by removing the tabular columns from validation dataset
X_images_val = x_valall.drop(['dx_type','age', 'sex' , 'localization'], axis=1).values

# Extracting the target labels for validation set
y_train_val = y_val.values

In [ ]:
# Split into image and tabular data for test dataset

# Extracting the tabular features (dx_type, age, sex, localization) from test dataset
X_tabular_test = x_testall[['dx_type','age', 'sex' , 'localization']]

# Extracting the image features by removing the tabular columns from testdataset
X_images_test = x_testall.drop(['dx_type','age', 'sex' , 'localization'], axis=1).values

# Extracting the target labels for test set
y_train_test = y_test.values

In [ ]:
x_train_imgs

In [ ]:
# show shape of images data
x_train_imgs.shape

In [ ]:
# Adding the target column to the dataset
tabular_data = x_train_tabilar
tabular_data['class'] = y_train.values
tabular_data

In [ ]:
# Ensure the similarity of the index
tabular_data.index == x_train_imgs.index

In [ ]:
# reset index of two datasets
tabular_data = tabular_data.reset_index(drop=True)
x_train_imgs = x_train_imgs.reset_index(drop = True)


In [ ]:
x_train_imgs

In [ ]:
# Check the shape of the tabular_data
tabular_data.shape

In [ ]:
# Check the shape of the x_train_imgs
x_train_imgs.shape

# Handle Unbalanced

In [ ]:
# oversampler = RandomOverSampler(random_state=42)


# # Apply oversampling
# x_train_resampled, y_train_resampled = oversampler.fit_resample(x_trainall, y_train)

In [ ]:
def reshape_to_image(flat_data):
    """
    Reshapes flat data into image format (height, width, channels).

    Parameters:
    flat_data: numpy array or pandas DataFrame
        The flattened data to be reshaped.

    Returns:
    numpy array
        Reshaped image data.
    """
    if isinstance(flat_data, pd.DataFrame):          # Check if the input data is a Pandas DataFrame
        flat_data = flat_data.values                 # Convert the DataFrame to a NumPy array
    return flat_data.reshape(-1, 28, 28, 3)          # Reshape the data to have dimensions (n_samples, 28, 28, 3)


# A function to reshape image data back to flat format
def reshape_to_flat(image_data):
    """
    Reshapes image data into flat format.

    Parameters:
    image_data: numpy array
        The image data to be reshaped.

    Returns:
    numpy array
        Flattened image data.
    """
    return image_data.reshape(image_data.shape[0], -1)    # Reshaping the image_data array to flatten each image


# A function to create an augmentation pipeline
def create_augmentation_pipeline():
    """
    Creates a data augmentation pipeline using albumentations.

    Returns:
    albumentations.Compose
        The composed transformation pipeline.
    """  
    # Define the sequence of transformations
    # These transformations help enhance the dataset by generating diverse variations of images To improve generalization.
    transform = A.Compose([
        A.Rotate(limit=30, p=0.7),      # Randomly rotates the image within +30 or -30 degrees with a 70% probability.
        A.HorizontalFlip(p=0.5),        # Flips the image horizontally with a 50% probability.
        A.VerticalFlip(p=0.5),          # Flips the image vertically with a 50% probability.
        A.OneOf([                   # Randomly applies one of the noise addition techniques with a 20% probability.
            A.GaussNoise(p=1),          # Adds random Gaussian noise to the image.
            A.MultiplicativeNoise(p=1),   # Adds random multiplicative noise to the image.
        ], p=0.2),
        A.OneOf([                   # Randomly applies one of the blurring techniques with a 20% probability.
            A.MotionBlur(p=1),                 # Introduces motion blur to simulate the effect of movement.
            A.MedianBlur(blur_limit=3, p=1),   # Applies a median blur with a kernel size of up to 3.
            A.GaussianBlur(blur_limit=3, p=1),   # Applies a Gaussian blur with a kernel size of up to 3.
        ], p=0.2),
        A.ColorJitter(       # Adjusts image brightness, contrast, saturation, and hue within specified limits
                             # to simulate varying lighting conditions with a 70% probability.
            brightness=0.2,         # Adjusts brightness by +20% or -20%.
            contrast=0.2,           # Adjusts contrast by +20% or -20%.
            saturation=0.2,         # Adjusts saturation by +20% or -20%.
            hue=0.1,                # Adjusts hue by +10% or -10%.
            p=0.7
        ),
    ])
    return transform     # Returns the composed augmentation pipeline.

In [ ]:
def augment_minority_classes(x_train, x_train_tabilar, y_train, target_samples=None):
 
    """
    Augments minority classes in the training dataset to balance class distributions.

    This function performs data augmentation on image data associated with minority classes 
    while preserving tabular data alignment. It increases the number of samples for each 
    class to match the size of the largest class or a specified target number of samples.

    Parameters:
    ----------
    x_train : pandas.DataFrame or numpy.ndarray
        The input training data containing image features, either as a flat DataFrame 
        or a NumPy array.
    
    x_train_tabilar : pandas.DataFrame
        The tabular data associated with each image sample.
    
    y_train : pandas.Series or numpy.ndarray
        The target labels for the training data.

    target_samples : int, optional
        The desired number of samples for each class. If not provided, the size of the 
        largest class is used.

    Returns:
    -------
    tuple
        A tuple containing:
        - x_aug (pandas.DataFrame): Augmented image data.
        - x_aug_row (pandas.DataFrame): Augmented tabular data.
        - y_aug (pandas.Series): Updated target labels.
"""
    if isinstance(y_train, pd.Series):
        y_train = y_train.values
        
    # Count the number of samples for each class.
    class_counts = Counter(y_train)

    # If target_samples is not provided, set it to the number of samples in the largest class.
    if target_samples is None:       
        target_samples = max(class_counts.values())
        
    # Convert the training data into image format if needed.
    x_images = reshape_to_image(x_train)
    
    # Scale pixel values to 0-255 range if they are normalized (values <= 1.0).
    if x_images.max() <= 1.0:
        x_images = (x_images * 255).astype(np.uint8)
        
    # call the data augmentation pipeline to apply the required transformation.
    transform = create_augmentation_pipeline()

    # Initialize lists to store augmented images and their corresponding labels and row data.
    augmented_images = []
    augmented_rows = []
    augmented_labels = []

    for class_label in class_counts:    # Iterate over each class in the dataset.
        
        # Initialize lists to store augmented images and their corresponding labels.
        class_indices = np.where(y_train == class_label)[0]
        class_images = x_images[class_indices]
        class_rows = x_train_tabilar.iloc[class_indices]  # Use .iloc to preserve positional indexing

        # Determine the number of samples for the current class.
        n_samples = len(class_images)
        if n_samples < target_samples:        # If the class is underrepresented, generate additional samples by augmentation.
            n_augment = target_samples - n_samples         # Number of new samples to create.

            # Perform data augmentation until the desired number of samples is reached.
            for i in range(n_augment):
                idx = i % n_samples
                image = class_images[idx].astype(np.uint8)

                # Apply the augmentation pipeline to the current image.
                augmented = transform(image=image)
                augmented_image = augmented['image']

                # Add the augmented image and corresponding label to the lists.
                augmented_images.append(augmented_image)
                augmented_labels.append(class_label)

                # Append tabular data as a proper row (ensure it's a 1D array, not 2D or nested)
                augmented_rows.append(class_rows.iloc[idx].to_numpy())

    # Combine original and augmented data, if any new samples were generated.
    if augmented_images:
        
        # Convert augmented images and rows to DataFrames
        augmented_images_df = pd.DataFrame(
            reshape_to_flat(np.array(augmented_images)), 
            columns=x_train.columns if isinstance(x_train, pd.DataFrame) else None
        )
        # Ensure whether it is a DataFrame.
        augmented_rows_df = pd.DataFrame(
            augmented_rows, 
            columns=x_train_tabilar.columns if isinstance(x_train_tabilar, pd.DataFrame) else None
        )
        augmented_labels_series = pd.Series(augmented_labels, name="label")

        # Concatenate the original and augmented data
        x_aug = pd.concat([x_train, augmented_images_df], ignore_index=True)
        x_aug_row = pd.concat([x_train_tabilar, augmented_rows_df], ignore_index=True)
        y_aug = pd.concat([pd.Series(y_train, name="label"), augmented_labels_series], ignore_index=True)

        return x_aug, x_aug_row, y_aug

    return x_train, x_train_tabilar, y_train     

# Apply augmentation to the training dataset.
x_train_aug,x_aug_row, y_train_aug = augment_minority_classes(x_train_imgs,tabular_data, y_train)

print("Before:")
print(Counter(y_train))
print("\nAfter:")
print(Counter(y_train_aug))

In [ ]:
# show row data after augmentation
x_aug_row

In [ ]:
# show image data after augmentation
x_train_aug

In [ ]:
# Visualizing the class distribution after augmentation
# Create a figure with a size of 6x6 inches.
plt.figure(figsize=(6, 6))

# Plot a pie chart using the counts of each label.
# Display percentages on the chart with one decimal point for each slice.
plt.pie(x_aug_row['class'].value_counts().values , labels=x_aug_row['class'].value_counts().index ,autopct='%1.1f%%')
plt.title('Classes pie chart')     # Add a title to the chart: "Classes pie chart".
plt.show()     # show the chart

In [ ]:
# Normalize pixel values
# Normalizing the pixel values of the training dataset (x_train_aug) by dividing by 255.0 to improve model convergence.
x_train_aug = x_train_aug / 255.0

# Normalizing the validation dataset (X_images_val) by dividing by 255.0.
X_images_val = X_images_val / 255.0

# Normalizing the test dataset (X_images_test) by dividing by 255.0.
X_images_test = X_images_test / 255.0

# Reshaping the augmented training data (x_train_aug) into a 4D array with shape (samples, height, width, channels),
# where -1 indicates an automatic adjustment to match the number of samples, 28x28 is the image size, 
# and 3 represents the color channels (RGB).
image_shape = (28, 28, 3)

x_train_aug = x_train_aug.to_numpy()  # Convert to NumPy array
x_train_aug_reshaped = x_train_aug.reshape(-1, *image_shape) # Reshape images


X_images_val_reshaped = X_images_val.reshape(-1, *image_shape)
X_images_test_reshaped = X_images_test.reshape(-1, *image_shape)

In [ ]:
# Encode gender column
X_tabular_resampled = x_aug_row 
lbl = LabelEncoder()  # create object from encoding class
X_tabular_resampled['sex_encoded'] = lbl.fit_transform(X_tabular_resampled['sex'])  # apply on train data
X_tabular_val['sex_encoded'] = lbl.transform(X_tabular_val['sex'])     # apply on validation data
X_tabular_test['sex_encoded'] = lbl.transform(X_tabular_test['sex'])   # apply on test data

# Encode dx_type column
X_tabular_resampled['dx_type_encoded'] = lbl.fit_transform(X_tabular_resampled['dx_type'])  # apply on train data
X_tabular_val['dx_type_encoded'] = lbl.transform(X_tabular_val['dx_type'])      # apply on validation data
X_tabular_test['dx_type_encoded'] = lbl.transform(X_tabular_test['dx_type'])    # apply on test data

# Encode localization column
X_tabular_resampled['localization_encoded'] = lbl.fit_transform(X_tabular_resampled['localization'])  # apply on train data
X_tabular_val['localization_encoded'] = lbl.transform(X_tabular_val['localization'])      # apply on validation data
X_tabular_test['localization_encoded'] = lbl.transform(X_tabular_test['localization'])    # apply on test data

In [ ]:
# Scale age column using minmaxscaler
scaler = MinMaxScaler()  # create object 
X_tabular_resampled['age_scaled'] = scaler.fit_transform(X_tabular_resampled[['age']])  # apply on train data
X_tabular_val['age_scaled'] = scaler.transform(X_tabular_val[['age']])                # apply on validation data
X_tabular_test['age_scaled'] = scaler.transform(X_tabular_test[['age']])            # apply on test data

In [ ]:
# Prepare tabular data by selecting relevant features from the resampled data
X_tabular_prepared_train = X_tabular_resampled[['age_scaled', 'localization_encoded','sex_encoded' , 'dx_type_encoded']].values
X_tabular_prepared_val = X_tabular_val[['age_scaled', 'localization_encoded','sex_encoded' ,'dx_type_encoded']].values
X_tabular_prepared_test = X_tabular_test[['age_scaled', 'localization_encoded','sex_encoded' ,'dx_type_encoded']].values

In [ ]:
# check its shape
X_tabular_prepared_train.shape

# Model

In [ ]:
# Image input branch with Convolutional layers
image_input = Input(shape= (28,28,3), name="image_input")

# Add convolutional and pooling layers
x = Conv2D(32, (3, 3), activation='relu', padding='same')(image_input)  # First Conv2D layer
x = MaxPooling2D((2, 2))(x)  # First pooling layer

x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)  # Second Conv2D layer
x = MaxPooling2D((2, 2))(x)  # Second pooling layer

x = Flatten()(x)  # Flatten the features for the dense layers
x = Dense(64, activation='relu')(x)
x = Dense(128, activation='relu')(x)  # Dense layer after flattening

# Tabular input branch
tabular_input = Input(shape=(4,), name="tabular_input")  # Adjust for 4 features
y = Dense(16, activation='relu')(tabular_input)  # Dense layer for tabular input

# Combine both branches
combined = Concatenate()([x, y])
z = Dense(128, activation='relu')(combined)  # Dense layer after combining
z = Dense(64, activation='softmax')(z)
z = Dense(32, activation='softmax')(z)
z = Dense(16, activation='softmax')(z)
z = Dense(7, activation='softmax')(z)  # Output layer for classification




In [ ]:
# Create the model
model = Model(inputs=[image_input, tabular_input], outputs=z)

# Compile the model with the Adam optimizer and sparse categorical crossentropy loss function
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Display model summary
model.summary()

In [ ]:
# List of callbacks to pass to the model during training

callbacks = [                     # EarlyStopping callback to stop training when validation loss stops improving
    EarlyStopping(
        monitor='val_accuracy',        # Monitor validation loss for early stopping
        patience=15,                   # Number of epochs to wait for improvement
        restore_best_weights=True,     # Restore the model weights from the best epoch
        verbose=1
    ),
    ModelCheckpoint(          # ModelCheckpoint callback to save the model with the best validation loss
        'best_model.keras',        # Filepath to save the best model
        monitor='val_accuracy',    # Monitor the validation loss metric
        save_best_only=True,       # Save only the best model based on validation loss
        verbose=1,                 
        mode='max'              # Since we're monitoring accuracy
    ),
    ReduceLROnPlateau(     # ReduceLROnPlateau: Reduces the learning rate by a factor of 0.5 if the validation loss does not improve for 5 epochs.
        monitor='val_loss',
        factor=0.5,             # Less aggressive reduction
        patience=5,
        min_lr=1e-6,
        verbose=1
    )
]

In [ ]:
# train the model
history = model.fit(
    [x_train_aug_reshaped, X_tabular_prepared_train], y_train_aug,
    validation_data=([X_images_val_reshaped , X_tabular_prepared_val], y_val),
    epochs=50,
    batch_size=32,
    callbacks = callbacks

)

# Evaluation

In [ ]:
# print model accurices on train & validation & test
train_score = model.evaluate([x_train_aug_reshaped, X_tabular_prepared_train], y_train_aug, verbose=1)
val_score = model.evaluate([X_images_val_reshaped , X_tabular_prepared_val], y_val, verbose=1)
test_score = model.evaluate([X_images_test_reshaped , X_tabular_prepared_test], y_test, verbose=1)

print("\nFinal Results:")
print(f"Train Accuracy: {train_score[1]*100:.2f}%")
print(f"Validation Accuracy: {val_score[1]*100:.2f}%")
print(f"Test Accuracy: {test_score[1]*100:.2f}%")


In [ ]:
# show loss & accuracy plot
plt.figure(figsize=(12, 4))

# Display the loss curve over the epochs.
plt.subplot(1, 2, 1)   # Creating the first subplot
plt.plot(history.history['loss'], label='Train Loss')   # Plotting the training loss
plt.plot(history.history['val_loss'], label='Validation Loss')    # Plotting the validation loss
plt.title('Model Loss')     # Adding a title to the loss plot
plt.xlabel('Epoch')      # Label for the x-axis (epochs)
plt.ylabel('Loss')       # Label for the y-axis (loss value)
plt.legend()           # Displaying the legend to differentiate the curves

# Display the accuracy curve over the epochs.
plt.subplot(1, 2, 2)      # Creating the second subplot
plt.plot(history.history['accuracy'], label='Train Accuracy')     # Plotting the training accuracy
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')    # Plotting the validation accuracy
plt.title('Model Accuracy')     # Adding a title to the accuracy plot
plt.xlabel('Epoch')         # Label for the x-axis (epochs)
plt.ylabel('Accuracy')      # Label for the y-axis (accuracy percentage)
plt.legend()        # Displaying the legend to differentiate the curves

# Adjusting the layout to ensure that the subplots do not overlap and are well-spaced.
plt.tight_layout()
plt.show()     # Displaying the plots.

In [ ]:
# save the model
model.save('truetrue_model.h5')  # Save the model in HDF5 format


In [ ]:
from tensorflow.keras.models import load_model

# Path to the saved model
model_path = "/kaggle/input/truetrue/tensorflow2/default/1/truetrue_model.h5"

# Load the model
model = load_model(model_path)

# Example prediction
predictions = model.predict([X_images_test_reshaped , X_tabular_prepared_test])
pred_labels = np.argmax(predictions, axis=1)



In [ ]:
# print classification report
print(classification_report(y_test, pred_labels))

In [ ]:
# show confusion matrix
beige_brown_palette = ["#D2B48C", "#C2B280", "#8B4513", "#6F4F28", "#3E2B1B"]

# Generating the confusion matrix by comparing the true labels (y_test) and the predicted labels (pred_labels).
cm = confusion_matrix(y_test, pred_labels)

# Defining a custom colormap using the previously defined beige and brown color palette.
cmap = ListedColormap(beige_brown_palette) 

# Creating the confusion matrix heatmap That is customized with the following properties:
# annot=True: displays the numbers inside the heatmap cells.
# fmt="d": formats the annotations as integers.
# cmap=beige_brown_palette: applies the custom color palette.
# cbar=False: removes the color bar.
# 'xticklabels' and 'yticklabels': set the x and y axis labels to the class labels.
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap=cmap, cbar=False, 
            xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

**Precision:**
precision measure the accuracy of positive predictions.it tells us, of all the cases the model predicted as positive, how many are actualy positive

**Recall:**
Recall measures how well the model detects positive cases. It tells us, of all the actual positive cases, how many were correctly predicted by the model.

**F1_Score:**
F1-Score is the harmonic mean of precision and recall. It is a balanced metric that considers both precision and recall.

In [ ]:
# Define a color palette using a list of color codes, which will be used to color the bars in the plots.
beige_brown_palette = ["#D2B48C", "#C2B280", "#8B4513", "#6F4F28", "#3E2B1B"]

# Extract the unique class labels from the 'dx' column of the DataFrame.
class_labels = df['dx'].unique()

# Calculate precision, recall, and F1-score for each class using the true and predicted labels.
# The 'average=None' argument to return scores for each class separately.
precision = precision_score(y_test, pred_labels, average=None)
recall = recall_score(y_test, pred_labels, average=None)
f1 = f1_score(y_test, pred_labels, average=None)

# Create a 2x2 grid of subplots to visualize the performance metrics.
# The figsize argument controls the size of the figure.
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot the precision scores for each class.
axes[0, 0].bar(class_labels, precision, color=beige_brown_palette[:len(class_labels)])
axes[0, 0].set_title('Precision')
axes[0, 0].set_ylabel('Precision Score')
axes[0, 0].tick_params(axis='x', rotation=45)

# Plot the recall scores for each class.
axes[0, 1].bar(class_labels, recall, color=beige_brown_palette[:len(class_labels)])
axes[0, 1].set_title('Recall')
axes[0, 1].set_ylabel('Recall Score')
axes[0, 1].tick_params(axis='x', rotation=45)

# Plot the F1-scores for each class.
axes[1, 0].bar(class_labels, f1, color=beige_brown_palette[:len(class_labels)])
axes[1, 0].set_title('F1-Score')
axes[1, 0].set_ylabel('F1-Score')
axes[1, 0].tick_params(axis='x', rotation=45)

# Hide the last subplot (bottom-right corner).
axes[1, 1].axis('off')

# Adjust the layout to ensure proper spacing between plots.
plt.tight_layout()
plt.show()
